In [15]:
%reset -f

import sys
sys.path.append("../data/exadata/parquet_dataset/query_tool")

import os
import pandas as pd
import numpy as np
import datetime
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.linear_model import LinearRegression
from query_tool import M100DataClient
from matplotlib import pyplot as plt
from models import LinearNN, StateEstimator

dataset_path = "../data/m100"
processed_path = "../data"
client = M100DataClient(dataset_path)



In [17]:
"""Cell purpose: pull desired features from plugins"""


#TODO: Merging method, working for columns that aren't timestamp,node,value
#For now I think it's necessary to do it this way because of memory issues
# def extract_features(feature_list, n_rows, columns, year_month):
#     df = pd.DataFrame()
#     for f in feature_list: 
#         query_df = client.query(f, 
#                       columns=columns,
#                       year_month=year_month)
#         
#         samples = np.random.randint(low = 0, high = query_df.shape[0], size = n_rows)
#         sampled_df = query_df if query_df.shape[0] < n_rows else query_df.iloc[samples] 
#         multi_idx= pd.MultiIndex.from_arrays([sampled_df["timestamp"].values, sampled_df["node"].values], names = ("timestamp", "node"))
#         data = {f: sampled_df["value"].values}
#         processed_df = pd.DataFrame(index = multi_idx, data=data)
#         if df.empty:
#             df = processed_df
#         else:
#             df = pd.concat([df, processed_df], join="outer")
#     return df

def extract_features(feature_list, n_rows, nodes_of_interest, columns, year_month):
    plugin_df = pd.DataFrame()
    node_dfs = []
    for f in feature_list: 
        query_df = client.query(f, 
                      columns=columns,
                      year_month=year_month)   
        
        for noi in nodes_of_interest:
            idxs = np.where(query_df["node"].to_numpy().astype(np.int32) == noi, True, False)
            node_data = query_df.iloc[idxs]
            node_data.drop("node", axis = "columns", inplace=True)
            node_data = node_data.pivot("value", index="timestamp")
            node_data.rename({"value": str(noi) + "_" + str(f)}, inplace=True)
            node_dfs.append(node_data)
            
    plugin_df = pd.concat(node_dfs, join="outer")
    return plugin_df


n_rows = 50
nodes_of_interest = [i for i in range(10)]
nodes_of_interest = [0]
year_month = ["22-02"]

#ipmi collects physical/hardware properties of nodes
ipmi_features = ["ambient", "gpu0_core_temp", "p0_core0_temp"]
ipmi_columns = ["timestamp", "node", "value"]
ipmi_df = extract_features(ipmi_features, n_rows, nodes_of_interest, ipmi_columns, year_month)
ipmi_df.to_csv(os.path.join(processed_path, "ipmi"))

#vertiv plugin collects data for AC cooling 
# vertiv_features = ["Supply_Air_Temperature"]
# vertiv_labels = ["Return_Air_Temperature"]
# vertiv_df = client.query(["Supply_Air_Temperature", "Return_Air_Temperature"],
#                   columns=['timestamp','value'],
#                   year_month=["22-02"])



# #ganglia collects metrics on utilization of hardware
# ganglia_df = client.query(["load_fifteen"],
#                   columns=['timestamp', "node", 'value'],
#                   year_month=["22-02"])

# logics collects power consumption data from equipment and devices
# logics_df = client.query(["Tot"],
#                   columns=['timestamp', "device", "panel", 'value'],
#                   year_month=["22-02"])

#schneider collects data on liquid cooling system
# schneider_df = client.query(["Tot"],
#                   columns=['timestamp', "node", 'value'],
#                   year_month=["22-02"])

#weather data
# weather_df = client.query(["temp"],
#                   columns=['timestamp', 'value'],
#                   year_month=["22-02"])


ipmi_df.head(100)


Retrieving data of type: float


C:\Users\olive\AppData\Local\Temp\ipykernel_23924\3287829056.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  node_data.drop("node", axis = "columns", inplace=True)


TypeError: DataFrame.pivot() takes 1 positional argument but 2 positional arguments (and 1 keyword-only argument) were given

In [ ]:

# feature_dfs = [ipmi_df, vertiv_df, weather_df]
feature_dfs = [ipmi_df, vertiv_df] #start w/ small one for now


In [ ]:
"""Cell purpose: preprocess and/or merge data as desired"""
df = pd.concat(plugin_dfs)
# print(merged_df["timestamp"].astype(np.int64) * 10e-9)
# merge_df["timestamp"] = merge_df["timestamp"].apply(lambda ts: ts.as_unit("ns"))
df.head(10)

splits = np.floor([.8, .1, .1] * len(df))
train_df = df.iloc[splits[0]]
val_df = df.iloc[splits[1]]
test_df = df.iloc[splits[2]]



In [ ]:
# Will have to make work with timestamp indexign later
class M100Data(Dataset):
    def __init__(self, feature_df, label_df, transform = None, label_transform = None):
        self.feature_df = feature_df
        self.label_df = label_df
        self.transform = transform

    def __len__(self):
        return len(self.df.shape[0])

    def __getitem__(self, idx):
        return self.feature_df.iloc[idx], self.label_df.iloc[idx]
    
train_ds = M100Data(df,)

In [ ]:
trainloader = DataLoader(train_ds, batch_size = 8, shuffle = True)
model = LinearNN()

In [ ]:
"""Cell purpose: train"""
# model = LinearRegression().fit(merged_df["air_temp"].to_numpy().reshape(-1,1), merged_df["ambient_temp"].to_numpy().reshape(-1,1))

In [ ]:
"""Cell purpose: inference"""
# preds = model.predict(test_df["air_temp"].to_numpy().reshape(-1,1))


In [ ]:
"""Cell purpose: benchmarking"""

# labels = test_df["ambient_temp"].to_numpy().reshape(-1,1)
# 
# 
# timesteps = [t for t in range(preds.shape[0])]
# colors = iter(plt.cm.rainbow(np.linspace(0, 0.5, 2)))
# plt.scatter(preds, labels, c=["blue", "red"])
# 
# plt.legend(loc="upper left")
# plt.xlabel("Time")
# plt.ylabel("Ambient temperature")
# plt.grid()
# plt.show()



In [ ]:
""
